In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\

import networkx as nx
import numpy as np
import tensorflow as tf

from keras import backend as K
from keras.layers import Dense
from keras.models import Model, Sequential


from TCGAMultiOmics.multiomics import MultiOmicsData
from moge.network.heterogeneous_network import HeterogeneousNetwork


Using TensorFlow backend.


#  Import network from file

In [2]:
import pickle

# WRITE
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0.pickle', 'wb') as file:
#     pickle.dump(network, file)

# READ
# with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0.pickle', 'rb') as file:
with open('/Users/jonny/Desktop/PycharmProjects/MultiOmicsGraphEmbedding/data/miRNA-mRNA_network.pickle', 'rb') as file:
    network = pickle.load(file)
    network.remove_extra_nodes()
#     network.node_list = network.all_nodes
#     node_list = network.node_list

In [3]:
# READ edgelists
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_test_edges.pickle', 'rb') as file:
    test_edges_dict = pickle.load(file)
    
with open('moge/data/lncRNA_miRNA_mRNA/miRNA-mRNA_network_test_05_val_01_seed_0_val_edges.pickle', 'rb') as file:
    val_edges_dict = pickle.load(file)

# Load training data

In [3]:
X, y = network.multi_omics_data.load_data(modalities=["MIR", "GE"])

In [4]:
X["MIR"].shape

(460, 1870)

In [5]:
transcripts = network.multi_omics_data.MIR.get_genes_info()["Mature sequence"]

/Users/jonny/anaconda3/lib/python3.6/site-packages/TCGAMultiOmics/genomic.py:549: FutureWarning: 'MiRBase ID' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  gene_info = gene_info.join(self.targetScan_family_df.groupby("MiRBase ID").first(), on="MiRBase ID",how="left")


In [6]:
transcripts[transcripts.notna()].map(len).describe()

count    1617.000000
mean       21.403834
std         1.644691
min        16.000000
25%        21.000000
50%        22.000000
75%        22.000000
max        27.000000
Name: Mature sequence, dtype: float64

In [7]:
transcripts

MiRBase ID
hsa-let-7a-1                            NaN
hsa-let-7a-2         CUGUACAGCCUCCUAGCUUUCC
hsa-let-7a-3                            NaN
hsa-let-7b           UGAGGUAGUAGGUUGUGUGGUU
hsa-let-7c           UGAGGUAGUAGGUUGUAUGGUU
hsa-let-7d           AGAGGUAGUAGGUUGCAUAGUU
hsa-let-7e           UGAGGUAGGAGGUUGUAUAGUU
hsa-let-7f-1         CUAUACAAUCUAUUGCCUUCCC
hsa-let-7f-2         CUAUACAGUCUACUGUCUUUCC
hsa-let-7g           UGAGGUAGUAGUUUGUACAGUU
hsa-let-7i           UGAGGUAGUAGUUUGUGCUGUU
hsa-mir-1-1                             NaN
hsa-mir-1-2                             NaN
hsa-mir-100          CAAGCUUGUAUCUAUAGGUAUG
hsa-mir-101-1                           NaN
hsa-mir-101-2                           NaN
hsa-mir-103a-1                          NaN
hsa-mir-103a-2      AGCUUCUUUACAGUGCUGCCUUG
hsa-mir-103b-1                          NaN
hsa-mir-103b-2                          NaN
hsa-mir-105-1                           NaN
hsa-mir-105-2                           NaN
hsa-mir-106a         

# Training Source Target Graph Embedding

In [43]:
from keras.layers import Input, Conv1D, Lambda, Dot, Dense, Flatten, MaxPooling1D, Lambda
from keras.layers import LSTM, Dense, TimeDistributed
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K

from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy

from keras.utils import to_categorical


def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = np.random.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
#//TODO: figure out how to initialize layer biases in keras.
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=np.random.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)


In [45]:
K.clear_session()
tf.reset_default_graph()
sess.close()
sess = tf.InteractiveSession()

/Users/jonny/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [46]:
# node_features_size = X["MIR"].shape[0]
input_shape = (None, 4)
_d = 128
# n_nodes = len(network.node_list)

In [77]:
# with tf.name_scope('inputs'):
E_ij = Input(batch_shape=(1, ), name="E_ij")
#     input_i = tf.placeholder(tf.float32, shape=input_shape, name="input_i")
#     input_j = tf.placeholder(tf.float32, shape=input_shape, name="input_j")
input_seq_i = Input(batch_shape=(1, None, 4), name="input_i")
input_seq_j = Input(batch_shape=(1, None, 4), name="input_j")
# is_directed = tf.placeholder(tf.bool, name="is_directed")
is_directed = Input(batch_shape=(1, ), dtype=tf.bool, name="is_directed")
i = tf.Variable(int, name="i", trainable=False)
j = tf.Variable(int, name="j", trainable=False)

In [78]:
#build convnet to use in each siamese 'leg'
lstm = Sequential()
lstm.add(LSTM(128, input_shape=(None, 4), return_sequences=False))
lstm.add(Dense(_d, activation='sigmoid'))

# encode each of the two inputs into a vector with the convnet
encoded_i = lstm(input_seq_i)
encoded_j = lstm(input_seq_j)
print("encoded_i", encoded_i, "\nencoded_j", encoded_j)


#layer to merge two encoded inputs with the l1 distance between them
# L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
# #call this layer on list of two input tensors.
# L1_distance = L1_layer([encoded_i, encoded_j])
# prediction = Dense(1, activation='sigmoid',bias_initializer=b_init)(L1_distance)


# # output = Lambda(lambda tup: 
# #                 is_directed*tup[0] + (1-is_directed)*tup[1])([output_directed, output_undirected])
# conditional_layer = Lambda(lambda x: K.switch(x[0], K.sigmoid(x[1]), K.sigmoid(x[2])))
# conditional_output = conditional_layer([is_directed, dot_directed, dot_undirected])
# print("conditional_output", conditional_output)

siamese_net = Model(inputs=[input_seq_i, input_seq_j, is_directed], outputs=[encoded_i, encoded_j])

encoded_i Tensor("sequential_6/dense_6/Sigmoid:0", shape=(1, 128), dtype=float32) 
encoded_j Tensor("sequential_6_1/dense_6/Sigmoid:0", shape=(1, 128), dtype=float32)


In [81]:
def contrastive_loss(encoded_i, encoded_j, E_ij, is_directed):
    dot_directed = Dot(axes=1)([encoded_i[:, 0:int(_d/2)], encoded_j[:, int(_d/2):_d]])
    dot_undirected = Dot(axes=1)([encoded_i, encoded_j])
    print("E_ij", E_ij, "is_directed", is_directed)
    print(dot_directed, dot_undirected)
    
    distance = -K.log(K.switch(is_directed, K.sigmoid(dot_directed), K.sigmoid(dot_undirected)))
    
    print(distance)

    loss = (1-E_ij)*1/2*K.square(distance) + E_ij*1/2*K.square(K.maximum(0.0, 0.5 - distance))
    print(loss)
    return loss

In [82]:
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss=contrastive_loss(encoded_i, encoded_j, E_ij, is_directed), 
                    optimizer=Adam(0.00006))

siamese_net.count_params()


E_ij Tensor("E_ij_5:0", shape=(1,), dtype=float32) is_directed Tensor("is_directed_5:0", shape=(1,), dtype=bool)
Tensor("dot_23/ExpandDims:0", shape=(1, 1), dtype=float32) Tensor("dot_24/ExpandDims:0", shape=(1, 1), dtype=float32)
Tensor("Neg_10:0", shape=(1, 1), dtype=float32)
Tensor("add_10:0", shape=(1, 1), dtype=float32)


TypeError: Using a `tf.Tensor` as a Python `bool` is not allowed. Use `if t is not None:` instead of `if t:` to test if a tensor is defined, and use TensorFlow ops such as tf.cond to execute subgraphs conditioned on the value of a tensor.

# Data Generator

In [83]:
from moge.network.data_generator import DataGenerator

In [86]:
generator = DataGenerator(network.node_list, network=network, 
                          batch_size=1, dim=(None, 4), shuffle=True)

IndexError: tuple index out of range

In [ ]:
siamese_net.fit_generator()

In [32]:
X, y = next(train_generator())
print(X.shape, y.shape)

(1, 88, 4) (1, 88, 1)


In [31]:
def train_generator():
    while True:
        sequence_length = np.random.randint(10, 100)
        x_train = np.random.random((1, sequence_length, 4))
        # y_train will depend on past 5 timesteps of x
        y_train = x_train[:, :, 0]
        for i in range(1, 2):
            y_train[:, i:] += x_train[:, :-i, i]
        y_train = to_categorical(y_train > 2.5)
        yield x_train, y_train

In [126]:
with tf.name_scope('siamese'):
    siamese = Dense(128, activation='relu')(N_i)
    siamese = Dense(_d, activation='relu')(siamese)
    
    emb_c_i = siamese(N_i)
    emb_c_i = siamese(N_j)


In [127]:
with tf.name_scope('embedding'):
    emb_s = tf.Variable(initial_value=tf.random_uniform([n_nodes, int(_d/2)], -1, 1),
                        validate_shape=True, dtype=tf.float32,
                        name="emb_t", trainable=True)

    emb_t = tf.Variable(initial_value=tf.random_uniform([n_nodes, int(_d/2)], -1, 1),
                        validate_shape=True, dtype=tf.float32,
                        name="emb_s", trainable=True)

    emb_c = tf.concat([emb_s, emb_t], axis=1, name="emb_concat")

In [128]:
emb_s = emb_s[i].assign(tf.reshape(siamese[:, 0 : int(_d/2)], [-1]))
emb_t = emb_t[i].assign(tf.reshape(siamese[:, int(_d/2) : _d], [-1]))

# with tf.control_dependencies([emb_s[i].assign(emb_c_i)]):
#     emb_s = tf.identity(emb_s)

In [ ]:
sess.as_default()
K.set_session(sess)
session.run(init_op)